# 初始化

In [1]:
import os
import gc
import time
import numpy as np  
np.set_printoptions(threshold=np.inf)
import tensorflow as tf
from keras import backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
K.set_session(sess)

from keras import layers
from keras.models import Model
from keras.layers import Input,Dense,Flatten,Dropout,ZeroPadding2D,BatchNormalization,Activation,add,AveragePooling2D
from keras.layers.convolutional import Conv2D,MaxPooling2D  
from keras.applications.imagenet_utils import preprocess_input

Using TensorFlow backend.


# 读取训练数据

In [2]:
dataset = 'AID'
base_path='./Dataset/' + dataset
ModelName = "GoogLeNet"
batch_size = 128
epochs = 50

X_Train = np.load(base_path + "/X_Train_224.npy");
Y_Train = np.load(base_path + "/Y_Train.npy");

print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + "Get %d Train Images" %(len(X_Train)))    
num_classes = Y_Train.shape[1]

2020-02-01 05:45:30Get 128000 Train Images


# GoogLeNet

In [3]:
def Conv2d_BN(x, nb_filter,kernel_size, padding='same',strides=(1,1),name=None):  
    if name is not None:  
        bn_name = name + '_bn'  
        conv_name = name + '_conv'  
    else:  
        bn_name = None  
        conv_name = None  
  
    x = Conv2D(nb_filter,kernel_size,padding=padding,strides=strides,activation='relu',name=conv_name)(x)  
    x = BatchNormalization(axis=3,name=bn_name)(x)  
    return x  

def Inception(x,nb_filter):  
    branch1x1 = Conv2d_BN(x,nb_filter,(1,1), padding='same',strides=(1,1),name=None)  
  
    branch3x3 = Conv2d_BN(x,nb_filter,(1,1), padding='same',strides=(1,1),name=None)  
    branch3x3 = Conv2d_BN(branch3x3,nb_filter,(3,3), padding='same',strides=(1,1),name=None)  
  
    branch5x5 = Conv2d_BN(x,nb_filter,(1,1), padding='same',strides=(1,1),name=None)  
    branch5x5 = Conv2d_BN(branch5x5,nb_filter,(1,1), padding='same',strides=(1,1),name=None)  
  
    branchpool = MaxPooling2D(pool_size=(3,3),strides=(1,1),padding='same')(x)  
    branchpool = Conv2d_BN(branchpool,nb_filter,(1,1),padding='same',strides=(1,1),name=None)  
  
    x = layers.concatenate([branch1x1,branch3x3,branch5x5,branchpool],axis=3)  
  
    return x  

from keras.optimizers import SGD  
def GoogLeNet(shape):
    inpt = Input(shape=shape)  
    #padding = 'same'，填充为(步长-1）/2,还可以用ZeroPadding2D((3,3))  
    x = Conv2d_BN(inpt,64,(7,7),strides=(2,2),padding='same')  
    x = MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)  
    x = Conv2d_BN(x,192,(3,3),strides=(1,1),padding='same')  
    x = MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)  
    x = Inception(x,64)#256  
    x = Inception(x,120)#480  
    x = MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)  
    x = Inception(x,128)#512  
    x = Inception(x,128)  
    x = Inception(x,128)  
    x = Inception(x,132)#528  
    x = Inception(x,208)#832  
    x = MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)  
    x = Inception(x,208)  
    x = Inception(x,256)#1024  
    x = AveragePooling2D(pool_size=(7,7),strides=(7,7),padding='same')(x)  
    x = Dropout(0.4)(x)  
    x = Flatten()(x)  
    x = Dense(1000,activation='relu')(x)  
    x = Dense(num_classes,activation='softmax')(x)  

    model = Model(inputs=inpt,outputs=x, name=ModelName)  
    sgd = SGD(decay=0.0001,momentum=0.9)  
    model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy']) 
    
    return model

In [4]:
model = GoogLeNet(X_Train.shape[1:])
model.summary()

Model: "GoogLeNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 112, 112, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 56, 56, 64)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________

In [5]:
history = model.fit(X_Train, Y_Train, batch_size=batch_size, epochs=epochs, shuffle=True)

Epoch 1/50
128000/128000 [==============================] - 8001s 63ms/step - loss: 1.0084 - accuracy: 0.6845
Epoch 2/50
128000/128000 [==============================] - 9323s 73ms/step - loss: 0.2746 - accuracy: 0.9073
Epoch 3/50
128000/128000 [==============================] - 11987s 94ms/step - loss: 0.1063 - accuracy: 0.9641
Epoch 4/50
128000/128000 [==============================] - 10926s 85ms/step - loss: 0.0508 - accuracy: 0.9835
Epoch 5/50
128000/128000 [==============================] - 8254s 64ms/step - loss: 0.0325 - accuracy: 0.9892
Epoch 6/50
128000/128000 [==============================] - 8196s 64ms/step - loss: 0.0186 - accuracy: 0.9943
Epoch 7/50
128000/128000 [==============================] - 6892s 54ms/step - loss: 0.0108 - accuracy: 0.9967
Epoch 8/50
128000/128000 [==============================] - 6460s 50ms/step - loss: 0.0088 - accuracy: 0.9974
Epoch 9/50
128000/128000 [==============================] - 5722s 45ms/step - loss: 0.0059 - accuracy: 0.9984
Epoch 10

KeyboardInterrupt: 

# 保存模型

In [6]:
save_folder = 'models/' + time.strftime("%Y-%m-%d", time.localtime()) + '/' + dataset
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

# serialize model to JSON
import pickle
model_json = model.to_json()
with open(os.path.join(save_folder, ModelName + ".json"), "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save(os.path.join(save_folder, ModelName + ".h5"))
#pickle.dump(history.history, open('history/UCMerced_LandUse/AlexNet.p','wb'))
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + "Saved Model to disk")

2020-02-02 11:56:56Saved Model to disk
